In [ ]:
!pip3 install pyarxiv
!pip3 install pybtex
!pip3 install dropbox
!pip3 install tqdm
!pip3 install pycolors2
!pip3 install pdf2image
!pip3 install Unidecode

In [1]:
import os
from bib_handling_code.processbib import read_bibfile
from bib_handling_code.processbib import save_to_file
import requests
import pandas as pd 
from get_biblatex import GetBiblatex
import string

In [2]:
path_diag_bib = os.path.join('script_data/', 'diag_ss.bib')
diag_bib_raw = read_bibfile(None, path_diag_bib) # I changed the code in such a way that IF I give a second argument, it uses the second argument as a full path

In [3]:
staff_id_dict = {'Bram van Ginneken': [8038506, 123637526],
'Francesco Ciompi': [143613202],
'Alessa Hering': [153744566],
'Henkjan Huisman': [34754023],
'Colin Jacobs': [2895994],
'Peter Koopmans': [34726383],
'Jeroen van der Laak': [145441238, 145388932],
'Geert Litjens': [145959882],
'James Meakin': [4960344],
'Keelin Murphy': [35730362],
'Ajay Patel': [2109170880, 2116215861],
'Cornelia Schaefer-Prokop': [1419819133, 1445069528, 1400632685],
'Matthieu Rutten': [2074975080, 2156546],
'Jos Thannhauser': [5752941],
"Bram Platel" : [1798137], 
"Nico Karssemeijer" : [1745574], 
"Clarisa Sanchez" : [144085811, 32187701], 
"Nikolas Lessman" : [2913408], 
"Jonas Teuwen" : [32649341, 119024451], 
"Rashindra Manniesing" : [2657081]}

In [4]:
staff_year_dict = {
'Bram van Ginneken':  {'start' : 1996, 'end': 9999},
'Francesco Ciompi':  {'start' : 2013, 'end': 9999},
'Alessa Hering':  {'start' : 2018, 'end': 9999},
'Henkjan Huisman':  {'start' : 1992, 'end': 9999},
'Colin Jacobs':  {'start' : 2010, 'end': 9999},
'Peter Koopmans':  {'start' : 2022, 'end': 9999},
'Jeroen van der Laak':  {'start' : 1991, 'end': 9999},
'Geert Litjens':  {'start' : 2016, 'end': 9999},
'James Meakin':  {'start' : 2017, 'end': 9999},
'Keelin Murphy':  {'start' : 2018, 'end': 9999},
'Ajay Patel':  {'start' : 2015, 'end': 9999},
'Cornelia Schaefer-Prokop':  {'start' : 2010, 'end': 9999},
'Matthieu Rutten':  {'start' : 2019, 'end': 9999},
'Jos Thannhauser': {'start' : 2022, 'end': 9999},
"Bram Platel" : {'start' : 2010,  'end' : 2019},
"Nico Karssemeijer" : {'start' : 1989, 'end' : 2022}, 
"Clarisa Sanchez" : {'start' : 2008, 'end' : 2021}, 
"Nikolas Lessman" : {'start' : 2019, 'end' : 2022}, 
"Jonas Teuwen" : {'start' : 2017, 'end' : 2020}, 
"Rashindra Manniesing" : {'start' : 2010, 'end' : 2021}
}

In [5]:
def find_new_ssids():
    staff_dict = {key: {'ids': staff_id_dict[key], 'years': staff_year_dict[key]} for key in staff_id_dict}
    all_staff_id_ss_data = []

    # get data for staff memb|er
    for idx, (staff_name, values) in enumerate(staff_dict.items()):
        staff_ids = values['ids']
        staff_start = values['years']['start']
        staff_end = values['years']['end']
        print(f'[{idx+1}/{len(staff_dict.items())}]: {staff_name}')
        for staff_id in staff_ids:
            print('\t\t', staff_id)
            staff_id_ss_data = []
            # get semantic scolar data
            url=f'https://api.semanticscholar.org/graph/v1/author/{staff_id}/papers?fields=year,title,authors,externalIds,citationCount&limit=500'
            r = requests.get(url)
            ss_staff_data = r.json()['data']

            # loop over semantic scolar data entries
            for ss_staff_entry in ss_staff_data:
                ss_id = None
                ss_title = None
                ss_doi = None
                ss_citations = None
                ss_year = None

                if 'paperId' in ss_staff_entry:
                    ss_id = ss_staff_entry['paperId']
                if 'title' in ss_staff_entry:
                    ss_title = ss_staff_entry['title']
                if 'DOI' in ss_staff_entry['externalIds']:
                    ss_doi = ss_staff_entry['externalIds']['DOI']
                if 'citationCount' in ss_staff_entry:
                    ss_citations = ss_staff_entry['citationCount']
                if 'year' in ss_staff_entry:
                    ss_year = ss_staff_entry['year']

                if ss_year != None:
                    if not staff_start <= ss_year <= staff_end:
                    # probably doesnt belong to DIAG, still captured via another staff member if also in the same paper
                        continue

                staff_id_ss_data.append([staff_id, staff_name, staff_start, staff_end, ss_year, ss_id, ss_title, ss_doi, ss_citations])
            all_staff_id_ss_data.extend(staff_id_ss_data)    
    ss_columns = ['staff_id', 'staff_name', 'staff_from', 'staff_till', 'ss_year', 'ss_id', 'title', 'doi', 'ss_citations']
    df_all_staff_id_ss_data = pd.DataFrame(all_staff_id_ss_data, columns=ss_columns)
    print('DONE')
    return df_all_staff_id_ss_data, df_all_staff_id_ss_data['ss_id'].tolist()

In [39]:
url=f'https://api.semanticscholar.org/graph/v1/author/{143613202}/papers?fields=year,title,authors,externalIds,citationCount&limit=500'
r = requests.get(url)
ss_staff_data = r.json()['data']

In [6]:
def return_current_ssids(bib_file):
    ss_ids=[]
    for entry in bib_file:
        if entry.type == 'string':
            continue
        if 'ss_id' in entry.fields:
            ss_ids.append(entry.fields['ss_id'].translate(str.maketrans('', '', string.punctuation)))
    return ss_ids

In [7]:
df, new_ss_ids = find_new_ssids()
current_ss_ids=return_current_ssids(diag_bib_raw)

[1/20]: Bram van Ginneken
		 8038506
		 123637526
[2/20]: Francesco Ciompi
		 143613202
[3/20]: Alessa Hering
		 153744566
[4/20]: Henkjan Huisman
		 34754023
[5/20]: Colin Jacobs
		 2895994
[6/20]: Peter Koopmans
		 34726383
[7/20]: Jeroen van der Laak
		 145441238
		 145388932
[8/20]: Geert Litjens
		 145959882
[9/20]: James Meakin
		 4960344
[10/20]: Keelin Murphy
		 35730362
[11/20]: Ajay Patel
		 2109170880
		 2116215861
[12/20]: Cornelia Schaefer-Prokop
		 1419819133
		 1445069528
		 1400632685
[13/20]: Matthieu Rutten
		 2074975080
		 2156546
[14/20]: Jos Thannhauser
		 5752941
[15/20]: Bram Platel
		 1798137
[16/20]: Nico Karssemeijer
		 1745574
[17/20]: Clarisa Sanchez
		 144085811
		 32187701
[18/20]: Nikolas Lessman
		 2913408
[19/20]: Jonas Teuwen
		 32649341
		 119024451
[20/20]: Rashindra Manniesing
		 2657081
DONE


In [8]:
print(f"Number of current ss_ids in diag_bib: {len(current_ss_ids)}")
print(f"Number of new ss_ids: {len(new_ss_ids)}")


Number of current ss_ids in diag_bib: 940
Number of new ss_ids: 2163


## ss_ids not in the diag_bib but not in found by our staff member search

In [9]:
set(current_ss_ids) - set(new_ss_ids)

{'056af49e670a8c5385d7c39977acd8d49de9fff0',
 '1e0bf0fbefbb0404c3c18740123a969091c67ef4',
 '23bcee96f8bb86f925bb2d3d0d3b0fb3d870fcee',
 '2662ff8ddd7135754bc72ca833eacfc1f307aeb1',
 '271cc27f0f6b1b4a87e43eed7c85df9c729b3279',
 '2cbfeefc3025a11beced3681bd0b358d44d4c0a8',
 '2f10f440accedc9e5cac27b58f606939d41b9c0b',
 '31f676889affcf561b077e661380e5fd945aabba',
 '35ee6606ec99b5bf282a0c5f400edbd16a6e22d9',
 '39a52867ea3d60ec4f28182e0e618b762d3d7909',
 '4992f23e29e711eec8b92e17ce35610da3758ecd',
 '57e693a3e04b766b142f72df5c6c6767620fb369',
 '608a5409afa9a5f535d5b3f4776b75ae92ec9e26',
 '66f70286594b27928fb0eea3b4a0f292261c292f',
 '6873c25b4466ae69a4755dd1f937bf0b82f6ad11',
 '699e59747454c422766132be28c2aa3c4c04de99',
 '6a340e97fd3c7c5f9f2f5401d1a2fb07c39fb8ed',
 '6b18013b756f365ee486e752272327a554ea841a',
 '7383ab9582f0e50753a0571f6e4d759f99f49d27',
 '766b44a5bc3a4ffb30dd1b4b4d76752e8c81698e',
 '7bfa78ee6a6d93bac31a1e29bc0552ca8e5a215c',
 '8a4482179ae334ba0dbd76060c30332a65f9861a',
 '93d384fe

There are 1631 unique semantic scholar ids found. The current bib file has 915 unique semantic scholar ids.
The intersection is 910 = that means that there are 5 semantic scholar ids in the current bib that are not found from the new search. Why is that?

In [10]:
found_items=set(new_ss_ids)-set(current_ss_ids)
print(f"Number of newly found items: {len(found_items)}")

Number of newly found items: 708


In [11]:
new_items = df[df['ss_id'].isin(found_items)]

In [12]:
new_items = new_items[new_items['ss_year']>=2015]
new_items = new_items.loc[new_items['doi'].notna()]
new_items = new_items.drop_duplicates(subset='doi', keep='first')

In [13]:
new_items

,staff_id,staff_name,staff_from,staff_till,ss_year,ss_id,title,doi,ss_citations
3,8038506,Bram van Ginneken,1996,9999,2023.0,4bcd672218ecec70473c84f6f1cc52c64031f3e5,Continual learning strategies for cancer-indep...,10.1016/j.media.2023.102755,4
6,8038506,Bram van Ginneken,1996,9999,2023.0,9ddb2f47695191553a3623ac33eddeb9c7e416cd,The STOIC2021 COVID-19 AI challenge: applying ...,10.48550/arXiv.2306.10484,1
8,8038506,Bram van Ginneken,1996,9999,2023.0,be95c8bbd8a4297d620b1c2644cf2a898603e355,Kidney abnormality segmentation in thorax-abdo...,10.48550/arXiv.2309.03383,0
9,8038506,Bram van Ginneken,1996,9999,2023.0,e738d94746b0f33b1c48737022ab186618d4d4cd,SimpleRad: Patient-Friendly Dutch Radiology Re...,10.1007/978-3-031-28241-6_18,0
10,8038506,Bram van Ginneken,1996,9999,2022.0,2d49fb82fd715a213e2149f9f08dd2fb4749b235,Challenges in digital medicine applications in...,10.1038/s41467-022-30728-3,8
...,...,...,...,...,...,...,...,...,...
2100,2657081,Rashindra Manniesing,2010,2021,2021.0,4202f56b0e22eedd6921d9b26519bc72f89cf4fd,Correction to: Incorporating radiomics into cl...,10.1007/s00330-021-07721-3,5
2102,2657081,Rashindra Manniesing,2010,2021,2021.0,571285fab9ced6a4d0bf3abd5f886d9a64e1790c,Traumatic Cerebral Microbleeds in the Subacute...,10.3174/ajnr.A7028,3
2103,2657081,Rashindra Manniesing,2010,2021,2021.0,92a2cb9804d3599cc1e8e81c512d0203c6d10da7,The radiological interpretation of possible mi...,10.1007/s00234-021-02839-z,2
2104,2657081,Rashindra Manniesing,2010,2021,2021.0,c859bd469080b82dc14db62e78d65ef5b5ffa686,Incorporating radiomics into clinical trials: ...,10.1007/s00330-020-07598-8,46


# Removing blacklisted items

In [16]:
blacklist = pd.read_csv("./script_data/blacklist.csv")
new_items = new_items[~new_items['doi'].isin(blacklist['doi'].unique().tolist())]
new_items

,staff_id,staff_name,staff_from,staff_till,ss_year,ss_id,title,doi,ss_citations
3,8038506,Bram van Ginneken,1996,9999,2023.0,4bcd672218ecec70473c84f6f1cc52c64031f3e5,Continual learning strategies for cancer-indep...,10.1016/j.media.2023.102755,4
6,8038506,Bram van Ginneken,1996,9999,2023.0,9ddb2f47695191553a3623ac33eddeb9c7e416cd,The STOIC2021 COVID-19 AI challenge: applying ...,10.48550/arXiv.2306.10484,1
8,8038506,Bram van Ginneken,1996,9999,2023.0,be95c8bbd8a4297d620b1c2644cf2a898603e355,Kidney abnormality segmentation in thorax-abdo...,10.48550/arXiv.2309.03383,0
12,8038506,Bram van Ginneken,1996,9999,2022.0,41f386d38567e38132525cad9bdc7da1ad6e8f1c,Automated COVID-19 Grading With Convolutional ...,10.1109/tai.2021.3115093,6
24,8038506,Bram van Ginneken,1996,9999,2021.0,979a9f247700d00ff2c3f0612d5eb001379f93c8,The Medical Segmentation Decathlon,10.1038/s41467-022-30695-9,369
...,...,...,...,...,...,...,...,...,...
2092,32649341,Jonas Teuwen,2017,2020,2018.0,9fca3cab7f8cf1071094591bc13ca53de528dac6,Automated lesion detection and segmentation in...,10.1117/12.2318326,33
2102,2657081,Rashindra Manniesing,2010,2021,2021.0,571285fab9ced6a4d0bf3abd5f886d9a64e1790c,Traumatic Cerebral Microbleeds in the Subacute...,10.3174/ajnr.A7028,3
2103,2657081,Rashindra Manniesing,2010,2021,2021.0,92a2cb9804d3599cc1e8e81c512d0203c6d10da7,The radiological interpretation of possible mi...,10.1007/s00234-021-02839-z,2
2104,2657081,Rashindra Manniesing,2010,2021,2021.0,c859bd469080b82dc14db62e78d65ef5b5ffa686,Incorporating radiomics into clinical trials: ...,10.1007/s00330-020-07598-8,46


In [48]:
dois = new_items['doi'].tolist()

In [49]:
ss_ids = new_items['ss_id'].tolist()

**Here: remove blacklist items**

Check dois: the first one refers to this publication https://www.semanticscholar.org/paper/Continual-learning-strategies-for-detection-of-node-B%C3%A1ndi-Balkenhol/4bcd672218ecec70473c84f6f1cc52c64031f3e5, it's already in diag.bib and should have a ss_id associated (it seems to be a one-to-one match). Why is it detected as a new item?

In [32]:
!pip3 install semanticscholar

In [33]:
# Code to get citations from semantic scholar. If there are multiple ss_ids, we should get the number of citations for each of them and sum the two (or more?) values. 
def get_citations(ss_id):
    from semanticscholar import SemanticScholar
    sch = SemanticScholar()
    paper = sch.get_paper(ss_id)
    return len(paper['citations'])

In [45]:
# Read the diag.bib file
cwd = os.getcwd()
parent_directory = os.path.abspath(os.path.join(cwd, ".."))
diag_bib_path = os.path.join(parent_directory, 'diag.bib')

with open(diag_bib_path, encoding="utf8") as bibtex_file:
    diag_bib = bibtex_file.read()

for i, el in enumerate(dois): #Make sure to only use unique dois
    citations = 0
    if isinstance(ss_ids[i], list):
        for ss_id in ss_ids[i]:
            citations += get_citations(ss_id)
    else:
        citations = get_citations(ss_ids[i])
    # Update Dre's code and add an extra argument for citations so that it uses the correct number when creating the bib item? 
    reader = GetBiblatex(doi=el, diag_bib=diag_bib, num_citations=citations)
    bibtext = reader.get_bib_text()
    # Code to add item to diag.bib ?
    if bibtext != 'empty':
        diag_bib = diag_bib + bibtext
